In [1]:
# !python -m pip install elasticsearch
# !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
# !tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
# !chown -R daemon:daemon elasticsearch-7.9.2

In [7]:
import os
from subprocess import Popen, PIPE, STDOUT
# es_server.kill()
es_server = Popen(['/opt/ml/elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
! sleep 30

In [3]:
# ! /opt/ml/elasticsearch-7.9.2/bin/elasticsearch-plugin install analysis-nori

In [4]:
# es_server.kill()

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')

In [3]:
es.info()

{'name': '640277a98ae6',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'JL4mg3SiSJudLDB6M5KTlQ',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
# es.indices.create(index = 'document_v3',
#                   body = {
#                       'settings':{
#                           'analysis':{
#                               'analyzer':{
#                                   'my_analyzer':{
#                                       "type": "custom",
#                                       'tokenizer':'nori_tokenizer',
#                                       'decompound_mode':'mixed',
#                                       'stopwords':'_korean_',
#                                       "filter": ["lowercase",
#                                                  "my_shingle_f",
#                                                  "nori_readingform",
#                                                  "nori_number"]
#                                   }
#                               },
#                               'filter':{
#                                   'my_shingle_f':{
#                                       "type": "shingle"
#                                   }
#                               }
#                           },
#                           'similarity':{
#                               'my_similarity':{
#                                   'type':'BM25',
#                               }
#                           }
#                       },
#                       'mappings':{
#                           'properties':{
#                               'title':{
#                                   'type':'text',
#                                   'analyzer':'my_analyzer',
#                                   'similarity':'my_similarity'
#                               },
#                               'text':{
#                                   'type':'text',
#                                   'analyzer':'my_analyzer',
#                                   'similarity':'my_similarity'
#                               },
#                               'document_id':{
#                                   'type':'text'
#                               }
#                           }
#                       }
#                   }
#                   )

In [10]:
es.indices.get('document_v3')

{'document_v3': {'aliases': {},
  'mappings': {'properties': {'document_id': {'type': 'text'},
    'text': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'},
    'title': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'}}},
  'settings': {'index': {'number_of_shards': '1',
    'provided_name': 'document_v3',
    'similarity': {'my_similarity': {'type': 'BM25'}},
    'creation_date': '1621409268631',
    'analysis': {'filter': {'my_shingle_f': {'type': 'shingle'}},
     'analyzer': {'my_analyzer': {'filter': ['lowercase',
        'my_shingle_f',
        'nori_readingform',
        'nori_number'],
       'decompound_mode': 'mixed',
       'type': 'custom',
       'stopwords': '_korean_',
       'tokenizer': 'nori_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'S2twchjZT5-XfTLE9bm_uw',
    'version': {'created': '7090299'}}}}}

In [4]:
import json
import pandas as pd

with open("input/data/wikipedia_documents.json", 'r') as f:
    wiki_data = pd.DataFrame(json.load(f)).transpose()

    
# with open("code/sparse-mecab-koreanwiki/wikipedia_documents_kor_v3.json", 'r') as f:
#     wiki_data = pd.DataFrame(json.load(f)).transpose()
    

In [12]:
# from tqdm import tqdm

# for num in tqdm(range(len(wiki_data))):
#     es.index(index='document_v3', body = {"title" : wiki_data['title'][num], "text" : wiki_data['text'][num], 'document_id' : wiki_data['document_id'][num]})

In [5]:
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)
from konlpy.tag import Mecab
# from rank_bm25 import BM25Okapi

In [6]:
org_dataset = load_from_disk("/opt/ml/input/data/train_dataset")
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
) # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*"*40, "query dataset", "*"*40)
print(full_ds)

Loading cached processed dataset at /opt/ml/input/data/train_dataset/train/cache-5864ec783ab77348.arrow
Loading cached processed dataset at /opt/ml/input/data/train_dataset/validation/cache-ec5c6075463087d0.arrow


**************************************** query dataset ****************************************
Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 4192
})


In [7]:
import re
from konlpy.tag import Mecab

mecab = Mecab()

# 질문에 특정적인 용어를 삭제하는 함수
def remove_q(query):
    stop = "|".join(
        "어느 무엇인가요 무엇 누가 누구인가요 누구인가 누구 어디에서 어디에 어디서 어디인가요 어디를 어디 언제 어떤 어떠한 몇 얼마 얼마나 뭐 어떻게 무슨 \?".split(
            " "
        )
    )
    rm = re.sub(stop, "", query).strip()
    return rm


# Retriever 에서 사용되는 토크나이저
def process_morphs_simple(query):
    rm = remove_q(query)
    pos = mecab.pos(rm)
    return [x[0] for x in pos if x[1][0] != "J" and x[1][0] != "E" and x[1][0] != "X"]

' '.join(process_morphs_simple('현대적 인사조직관리의 시발점이 된 책은?'))

'현대 인사 조직 관리 시발점 된 책'

In [8]:
from pororo import Pororo
mrc = Pororo("mrc", "ko")

In [9]:
count = [0, 0, 0, 0, 0]
topk = [1, 3, 5, 10, 20]
check = []
c = 0

from tqdm import tqdm

# for i in range(4):
for j in tqdm(range(200)):
    question = full_ds['question'][j]
    ground_truth = full_ds['context'][j]
    ground_truth_did = full_ds['document_id'][j]
    
    query = {
        'query':{
            'bool':{
                'must':[
                          {'match':{'text':question}}
                ],
                'should':[
                          {'match':{'text':question}}
                ]
            }
        }
    }

    doc = es.search(index='document_v3',body=query,size=5)['hits']['hits']
    topk_contexts = []
    topk_contexts_id = []
    
    for i in range(len(doc)):
        topk_contexts.append(doc[i]['_source']['text'])
        topk_contexts_id.append(doc[i]['_source']['document_id'])
    
    answers = []
    scores = []
    top_index = -1
    for idx, text in enumerate(topk_contexts):
        score = 0
        answer = mrc(question, text)
        for element in answer:
            score += element[2]
        answers.append(answer)
        scores.append(score)
        
        if ground_truth_did == topk_contexts_id[idx]:
            top_index = idx
            
    print(top_index, scores)
    if top_index == scores.index(max(scores)):
        c+=1
#     for idx, text in enumerate(topk_contexts):
#         if ground_truth == text:
#             check.append(idx)
#             if idx == 0:
#                 count[0] += 1
#             elif 0 < idx < 3:
#                 count[1] += 1
#             elif 0 < idx < 5:
#                 count[2] += 1
#             elif 5 < idx < 10:
#                 count[3] += 1
#             elif 10 < idx < 20:
#                 count[4] += 1
#             break

# count[4] += (count[0] + count[1] + count[2] + count[3])
# count[3] += (count[0] + count[1] + count[2])
# count[2] += (count[0] + count[1])
# count[1] += count[0]

  0%|          | 1/200 [00:00<02:41,  1.23it/s]

0 [539.831646502018, 360.3888654783368, 68.74030423164368, 50.477137207984924, 7.460983991622925]


  1%|          | 2/200 [00:01<02:17,  1.44it/s]

0 [708.3188933134079, 38.210346188396215, 49.50748769938946, 9.703505396842957, 111.14139368245378]


  2%|▏         | 3/200 [00:01<02:07,  1.54it/s]

...

0 [1384.0206178724766, 258.4261407852173, 56.36170959472656, 343.9171689748764, 282.2228991985321]


100%|█████████▉| 199/200 [01:38<00:00,  1.88it/s]

1 [645.4346448779106, 645.4346448779106, 132.25774556398392, 160.20970630645752, 93.95533069968224]


100%|██████████| 200/200 [01:39<00:00,  2.02it/s]

0 [667.0761170387268, 562.8091044425964, 134.65399292111397, 39.95235422253609, 237.06180834770203]


In [10]:
c

83

In [ ]:
import numpy as np

print(count)
print(np.array(count) / len(full_ds))

In [ ]:
# !pip install matplotlib

import matplotlib.pyplot as plt

plt.hist(check, rwidth=0.8, bins=np.arange(0,20,1))
plt.show()